## How much heat are we adding to the ocean?

In [1]:
import cartopy.crs as ccrs
import cosima_cookbook as cc
import cartopy.feature as cfeature
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import cmocean as cm
from dask.distributed import Client
import matplotlib.path as mpath
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr

# Avoid the Runtime errors in true_divide encountered when trying to divide by zero
import warnings
warnings.filterwarnings('ignore', category = RuntimeWarning)
warnings.filterwarnings('ignore', category = ResourceWarning)
warnings.filterwarnings('ignore', category = BytesWarning)

# matplotlib stuff:
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib
from mpl_toolkits.mplot3d import axes3d
%matplotlib inline
matplotlib.rcParams['mathtext.fontset'] = 'stix'
matplotlib.rcParams['font.family'] = 'STIXGeneral'
matplotlib.rcParams['lines.linewidth'] = 2.0

In [2]:
import logging
logger = logging.getLogger("distributed.utils_perf")
logger.setLevel(logging.ERROR)

In [3]:
from dask.distributed import Client
client = Client()
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/33789/status,
Dashboard: /proxy/33789/status,Workers: 4
Total threads: 16,Total memory: 251.21 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:44727,Workers: 4
Dashboard: /proxy/33789/status,Total threads: 16
Started: Just now,Total memory: 251.21 GiB
Comm: tcp://127.0.0.1:39993,Total threads: 4
Dashboard: /proxy/34187/status,Memory: 62.80 GiB
Nanny: tcp://127.0.0.1:41251,


In [4]:
db = '/scratch/x77/db6174/mom/archive/databases/025deg_jra55_ryf/cc_database_flux_forced_3hourly.db'
session = cc.database.create_session(db)
expt = '025deg_jra55_ryf_flux_forced_3hourly'

In [5]:
start_time = '2099-01-01'
end_time = '2100-12-31'

In [6]:
sw_flux   = cc.querying.getvar(expt = expt, session = session, variable = 'swflx', frequency = '1 monthly').sel(time = slice(start_time, end_time))
sw_flux = sw_flux.mean('time')
lw_heat   = cc.querying.getvar(expt = expt, session = session, variable = 'lw_heat', frequency = '1 monthly').sel(time = slice(start_time, end_time))
lw_heat = lw_heat.mean('time')
sens_heat = cc.querying.getvar(expt = expt, session = session, variable = 'sens_heat', frequency = '1 monthly').sel(time = slice(start_time, end_time))
sens_heat = sens_heat.mean('time')
evap_heat = cc.querying.getvar(expt = expt, session = session, variable = 'evap_heat', frequency = '1 monthly').sel(time = slice(start_time, end_time))
evap_heat = evap_heat.mean('time')
frazil    = cc.querying.getvar(expt = expt, session = session, variable = 'frazil_3d_int_z', frequency = '1 monthly').sel(time = slice(start_time, end_time))
frazil = frazil.mean('time')

area_t = cc.querying.getvar(expt = expt, variable = 'area_t', session = session, frequency = 'static', n = 1)

### a. Total heat input

In [7]:
diff = sw_flux + lw_heat + sens_heat + evap_heat + frazil
diff = ((diff * area_t).sum(dim = ['xt_ocean', 'yt_ocean']))/area_t.sum(dim = ['xt_ocean', 'yt_ocean'])
diff.values

array(1.1014372, dtype=float32)

### b. Total heat input without frazil

In [8]:
diff_wo_frazil = sw_flux + lw_heat + sens_heat + evap_heat
diff_wo_frazil = ((diff_wo_frazil * area_t).sum(dim = ['xt_ocean', 'yt_ocean']))/area_t.sum(dim = ['xt_ocean', 'yt_ocean'])
diff_wo_frazil.values

array(0.86604965, dtype=float32)